In [1]:
import pandas as pd
import numpy as np

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, mean_squared_error, log_loss, f1_score,f1_score, accuracy_score,roc_auc_score,auc
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import gc

In [2]:
def load_train():
    df_train = pd.read_csv('data/gaiic_track3_round1_train_20210228.tsv',sep='\t',header=None)
    df_train.columns=['q1','q2','label']
    df_train['q1'] = df_train['q1'].fillna('好的')
    df_train['q2'] = df_train['q2'].fillna('好的')
    df_test =  pd.read_csv('data/gaiic_track3_round1_testB_20210317.tsv',sep='\t',header=None)
    df_test.columns=['q1','q2']
    df_test['q1'] = df_test['q1'].fillna('好的')
    df_test['q2'] = df_test['q2'].fillna('好的')
    train_id = np.arange(0,len(df_train),1)
    df_train['id'] =train_id
    test_id = np.arange(0,len(df_test),1)
    df_test['id'] =test_id 
    return df_train, df_test

def load_test_valid(score):
    valid = np.load(f'./B/oof_id_{score}.npy')
    test = np.load(f'./B/sub_id_{score}.npy')
    return valid,test

def word_match_share(q1words, q2words):
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    shared_words_in_q1 = [w for w in q1words if w in q2words]
    shared_words_in_q2 = [w for w in q2words if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

def word_match_share_set(q1words, q2words):
    if len(q1words) == 0 or len(q2words) == 0:
        return 0
    q1 = set(q1words)
    q2 = set(q2words)
    R = len(q1&q2)/(len(q1) + len(q2))
    R1 = len(q1&q2)/len(q1) 
    R2 = len(q1&q2)/len(q2) 
    return R,R1,R2



In [3]:
def diff_num_helper(df, time_col='id_sub', col = 'id', num_col='any_x'):
    df['ke_cnt_' + col] = df.groupby(col)[time_col].rank(ascending=False,method = 'first')
    #print(df)
    df2 = df[[col, 'ke_cnt_' + col, num_col]].copy()
    df2['ke_cnt_' + col] = df2['ke_cnt_' + col] - 1
    df3 = pd.merge(df, df2, on=[col, 'ke_cnt_' + col], how='left')
    df['LAG_{}_{}'.format(col, num_col)] = abs(df3[num_col +'_x'] - df3[num_col + '_y'])
    df['LAG_raw_{}_{}'.format(col, num_col)] = df3[num_col + '_y']
    del df2,df3,df['ke_cnt_' + col] 
    gc.collect()
    return df#[['DIFF_{}_{}'.format(col, num_col)]]

def diff_num_helper2(df, time_col='id_sub', col = 'id', num_col='any_x'):
    df['ke_cnt_' + col] = df.groupby(col)[time_col].rank(ascending=False,method = 'first')
    df2 = df[[col, 'ke_cnt_' + col, num_col]].copy()
    df2['ke_cnt_' + col] = df2['ke_cnt_' + col] + 1
    df3 = pd.merge(df, df2, on=[col, 'ke_cnt_' + col], how='left')
    assert df3[num_col +'_x'][0] == df[num_col][0]
    assert df3[num_col +'_y'][0] != df[num_col][0]

    df['LAG2_{}_{}'.format(col, num_col)] = abs(df3[num_col +'_x'] - df3[num_col + '_y'])
    df['LAG2_raw_{}_{}'.format(col, num_col)] = df3[num_col + '_y']
    #df['ke_cnt_ratio' + col] = df['ke_cnt_' + col] 
    del df2,df3,df['ke_cnt_' + col] 
    gc.collect()
    return df
def left_merge(data1, data2, on):
    if type(on) != list:
        on = [on]
    if (set(on) & set(data2.columns)) != set(on):
        data2_temp = data2.reset_index()
    else:
        data2_temp = data2.copy()
    columns = [f for f in data2.columns if f not in on]
    result = data1.merge(data2_temp,on=on,how='left')
    result = result[columns]
    return result

def agg_helper(main_df, df, num_col, col, table_name='test'):
    agg_dict = []
    agg_dict.append(('AGG_min_{}_{}_{}'.format(table_name, num_col, col),'min'))
    agg_dict.append(('AGG_max_{}_{}_{}'.format(table_name, num_col, col),'max'))
    agg_dict.append(('AGG_mean_{}_{}_{}'.format(table_name, num_col, col),'mean'))
    agg_dict.append(('AGG_median_{}_{}_{}'.format(table_name, num_col, col),'median'))
    agg_dict.append(('AGG_var_{}_{}_{}'.format(table_name, num_col, col),'var'))
    
    agg_result = df.groupby(col)[num_col].agg(agg_dict)
    merget_result = left_merge(main_df[[col]], agg_result, on = [col])
    return merget_result[[i for i in merget_result.columns if i != col]]
def agg_helper2(main_df, df, num_col, col, table_name='test'):
    agg_dict = {}
    agg_dict['AGG_nunique_{}_{}_{}'.format(table_name, num_col, col)] = 'nunique'
    agg_result = df.groupby(col)[num_col].count().reset_index()#agg(agg_dict)
    agg_result.columns=[col,  'AGG_nunique_{}_{}_{}'.format(table_name, num_col, col)]
    merget_result = left_merge(main_df[[col]], agg_result, on = [col])
    return merget_result[[i for i in merget_result.columns if i != col]]

def agg_helper3(main_df, df, num_col, col, table_name='test'):
    agg_result = df.groupby(col)[num_col].count().reset_index()#agg(agg_dict)
    agg_result.columns=[col,  'AGG_count_{}_{}_{}'.format(table_name, num_col, col)]
    merget_result = left_merge(df[[col]], agg_result, on = [col])
    return merget_result[[i for i in merget_result.columns if i != col]]

from tqdm import tqdm

def lable_encode(train):
    cat_col = []
    for feature in tqdm(train.columns):
        if train[feature].dtypes == 'object':
            encoder = LabelEncoder()    
            train[feature] = encoder.fit_transform(train[feature].astype(str))
            train[feature + '_count'] = train[feature].map(pd.value_counts(train[feature]))

            cat_col.append(feature)
    return train, cat_col

def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [4]:
all_train, all_test = load_train()
all_test['id'] = all_test['id']+1000000

In [5]:
df_test = all_test

In [6]:
all_train['concat'] = all_train['q1'] + all_train['q2']
all_test['concat'] = all_test['q1'] + all_test['q2']
same = all_test[['concat']].merge(all_train[['concat','label']].drop_duplicates(subset=['concat']).reset_index(drop=True),on=['concat'],how='left')

In [7]:
len(all_train)
len(all_test)

25000

In [8]:
# all_train['f1'], all_test['f1']  = load_test_valid('nezha-base_0.9731_100epoch')
# all_train['f2'], all_test['f2']  = load_test_valid('nezha-base_0.9744_500epoch')
# all_train['f3'], all_test['f3'] = load_test_valid('nezha-base_0.9750_300epoch')
# all_train['f4'], all_test['f4'] = load_test_valid('nezha-base_0.9762')
# all_train['f5'], all_test['f5'] = load_test_valid('nezha-base_0.9766_z')
# all_train['f6'], all_test['f6']  = load_test_valid('nezha-large_0.9728')
# all_train['f7'], all_test['f7']  = load_test_valid('nezha-base_0.9750660632375358')
# all_train['f8'], all_test['f8']  = load_test_valid('nezha-large_0.9681657612090724')
# all_train['f8'], all_test['f8']  = load_test_valid('nezha-large_0.970280295812428')
# all_train['f9'], all_test['f9']  = load_test_valid('nezha-large_0.9722644834299695_test')


# all_train['f1'], all_test['f1']  = load_test_valid('nezha-base_0.9591154881244821')
# # all_train['f2'], all_test['f2']  = load_test_valid('nezha-large_0.970280295812428')
# all_train['f3'], all_test['f3'] = load_test_valid('nezha-base_0.9731846431788207')
# all_train['f4'], all_test['f4'] = load_test_valid('nezha-base_0.9744678930253807')
# # all_train['f5'], all_test['f5'] = load_test_valid('nezha-base_0.9748702884103849')
# # all_train['f6'], all_test['f6']  = load_test_valid('nezha-base_0.9748813528826525')
# # all_train['f7'], all_test['f7']  = load_test_valid('nezha-base_0.9750660632375358')
# # all_train['f8'], all_test['f8']  = load_test_valid('nezha-large_0.9681657612090724')
# # all_train['f8'], all_test['f8']  = load_test_valid('nezha-large_0.970280295812428')
# all_train['f9'], all_test['f9']  = load_test_valid('nezha-large_0.9722644834299695_test')


# all_train['f11'], all_test['f11']  = load_test_valid(0.7748161764705883)
# all_train['f12'], all_test['f12']  = load_test_valid(0.7794956745812627)
# all_train['f13'], all_test['f13']  = load_test_valid(0.7826952665399584)
# all_train['f14'], all_test['f14']  = load_test_valid(0.7878124429848568)
# all_train['f15'], all_test['f15']  = load_test_valid(0.786074047892551)
# all_train['f16'], all_test['f16']  = load_test_valid(0.7840989399293286)
# all_train['f17'], all_test['f17']  = load_test_valid(0.7709487225299266)


# #加上这些只有825
# all_train['f18'], all_test['f18']  = load_test_valid(0.7886281588447653)
# all_train['f19'], all_test['f19']  = load_test_valid(0.7897426701335963)
# all_train['f20'], all_test['f20']  = load_test_valid(0.7827964729170417)
# all_train['f21'], all_test['f21']  = load_test_valid(0.7857014871886758)
# all_train['f22'], all_test['f22']  = load_test_valid(0.7844938453058976)
# all_train['f23'], all_test['f23']  = load_test_valid(0.7804581742450537)
# all_train['f24'], all_test['f24']  = load_test_valid(0.782546749777382)
# all_train['f25'], all_test['f25']  = load_test_valid(0.7877678080937557)



# #加上1020
# all_train['f26'], all_test['f26']  = load_test_valid(0.7944018058690745)
# all_train['f27'], all_test['f27']  = load_test_valid(0.7876785714285715)
# all_train['f28'], all_test['f28']  = load_test_valid(0.7879113018597997)
# all_train['f29'], all_test['f29']  = load_test_valid(0.7897916854540535)
# all_train['f30'], all_test['f30']  = load_test_valid(0.7853204374943507)
# all_train['f31'], all_test['f31']  = load_test_valid(0.7942125149755783)
# all_train['f32'], all_test['f32']  = load_test_valid(0.7813676560721479)

# #10月22

# all_train['f33'], all_test['f33']  = load_test_valid(0.7812527686719235)
# all_train['f34'], all_test['f34']  = load_test_valid(0.7922124528979007)
# all_train['f35'], all_test['f35']  = load_test_valid(0.7908595911809336)
# all_train['f36'], all_test['f36']  = load_test_valid(0.7952286282306162)
# all_train['f37'], all_test['f37']  = load_test_valid(0.7882094148702156)
# all_train['f38'], all_test['f38']  = load_test_valid(0.7835385030101536)
# all_train['f39'], all_test['f39']  = load_test_valid(0.7883380884450784)
# #10月23
# all_train['f40'], all_test['f40']  = load_test_valid(0.7840500966939865)
# all_train['f41'], all_test['f41']  = load_test_valid(0.7875834686879625)
# all_train['f42'], all_test['f42']  = load_test_valid(0.7898375078552832)
# all_train['f43'], all_test['f43']  = load_test_valid(0.7954884414615958)
# all_train['f44'], all_test['f44']  = load_test_valid(0.7875421182041709)
# all_train['f45'], all_test['f45']  = load_test_valid(0.7831971174971438)
# all_train['f46'], all_test['f46']  = load_test_valid(0.7862834631569523)
# all_train['f47'], all_test['f47']  = load_test_valid(0.7976395823876533)
# ##10yue 23r日
# all_train['f48'], all_test['f48']  = load_test_valid(0.7861343788985922)
# all_train['f49'], all_test['f49']  = load_test_valid(0.7813665726503503)
# all_train['f50'], all_test['f50']  = load_test_valid(0.7873311107211791)



In [9]:
test = np.load(f'B/sub.npy')
valid = np.load(f'B/oof.npy')

In [10]:
v1=[]
t1=[]
v2=[]
t2=[]
v3=[]
t3=[]
v4=[]
t4=[]
v5=[]
t5=[]
v6=[]
t6=[]
v7=[]
t7=[]
v8=[]
t8=[]
v9=[]
t9=[]
v10=[]
t10=[]
v11=[]
t11=[]
v12=[]
t12=[]
v13=[]
t13=[]
v14=[]
t14=[]
v15=[]
t15=[]
v16=[]
t16=[]
v17=[]
t17=[]
v18=[]
t18=[]
v19=[]
t19=[]
v20=[]
t20=[]
v21=[]
t21=[]
v22=[]
t22=[]
v23=[]
t23=[]
v24=[]
t24=[]
v25=[]
t25=[]
v26=[]
t26=[]
v27=[]
t27=[]
v28=[]
t28=[]
v29=[]
t29=[]
v30=[]
t30=[]
v31=[]
t31=[]
v32=[]
t32=[]
v33=[]
t33=[]
v34=[]
t34=[]
v35=[]
t35=[]
v36=[]
t36=[]
v37=[]
t37=[]
v38=[]
t38=[]
v39=[]
t39=[]
v40=[]
t40=[]
v41=[]
t41=[]
v42=[]
t42=[]
v43=[]
t43=[]
v44=[]
t44=[]
v45=[]
t45=[]
for i in range(len(valid)):
    v1.append(valid[i][0])
    v2.append(valid[i][1])
    v3.append(valid[i][2])
    v4.append(valid[i][3])
    v5.append(valid[i][4])
    v6.append(valid[i][5])
    v7.append(valid[i][6])
    v8.append(valid[i][7])
    v9.append(valid[i][8])
    v10.append(valid[i][9])
    v11.append(valid[i][10])
    v12.append(valid[i][11])
    v13.append(valid[i][12])
    v14.append(valid[i][13])
    v15.append(valid[i][14])
    v16.append(valid[i][15])
    v17.append(valid[i][16])
    v18.append(valid[i][17])
    v19.append(valid[i][18])
    v20.append(valid[i][19])
    v21.append(valid[i][20])
    v22.append(valid[i][21])
    v23.append(valid[i][22])
    v24.append(valid[i][23])
    v25.append(valid[i][24])
    v26.append(valid[i][25])
    v27.append(valid[i][26])
    v28.append(valid[i][27])
    v29.append(valid[i][28])
    v30.append(valid[i][29])
    v31.append(valid[i][30])
    v32.append(valid[i][31])
    v33.append(valid[i][32])
    v34.append(valid[i][33])
    v35.append(valid[i][34])
    v36.append(valid[i][35])
    v37.append(valid[i][36])
    v38.append(valid[i][37])
    v39.append(valid[i][38])
    v40.append(valid[i][39])
    v41.append(valid[i][40])
    v42.append(valid[i][41])
    v43.append(valid[i][42])
    v44.append(valid[i][43])
    v45.append(valid[i][44])
    

for i in range(len(test)):
    t1.append(test[i][0])
    t2.append(test[i][1])
    t3.append(test[i][2])
    t4.append(test[i][3])
    t5.append(test[i][4])
    t6.append(test[i][5])
    t7.append(test[i][6])
    t8.append(test[i][7])
    t9.append(test[i][8])
    t10.append(test[i][9])
    t11.append(test[i][10])
    t12.append(test[i][11])
    t13.append(test[i][12])
    t14.append(test[i][13])
    t15.append(test[i][14])
    t16.append(test[i][15])
    t17.append(test[i][16])
    t18.append(test[i][17])
    t19.append(test[i][18])
    t20.append(test[i][19])
    t21.append(test[i][20])
    t22.append(test[i][21])
    t23.append(test[i][22])
    t24.append(test[i][23])
    t25.append(test[i][24])
    t26.append(test[i][25])
    t27.append(test[i][26])
    t28.append(test[i][27])
    t29.append(test[i][28])
    t30.append(test[i][29])
    t31.append(test[i][30])
    t32.append(test[i][31])
    t33.append(test[i][32])
    t34.append(test[i][33])
    t35.append(test[i][34])
    t36.append(test[i][35])
    t37.append(test[i][36])
    t38.append(test[i][37])
    t39.append(test[i][38])
    t40.append(test[i][39])
    t41.append(test[i][40])
    t42.append(test[i][41])
    t43.append(test[i][42])
    t44.append(test[i][43])
    t45.append(test[i][44])
    
all_train['f1'],all_test['f1']  =v1,t1
all_train['f2'],all_test['f2']  =v2,t2,
all_train['f3'],all_test['f3']  =v3,t3,
all_train['f4'],all_test['f4']  =v4,t4,
all_train['f5'],all_test['f5']  =v5,t5,
all_train['f6'],all_test['f6']  =v6,t6,
all_train['f7'],all_test['f7']  =v7,t7,
all_train['f8'],all_test['f8']  =v8,t8,
all_train['f9'],all_test['f9']  =v9,t9,
all_train['f10'],all_test['f10']  =v10,t10,
all_train['f11'],all_test['f11']  =v11,t11,
all_train['f12'],all_test['f12']  =v12,t12,
all_train['f13'],all_test['f13']  =v13,t13,
all_train['f14'],all_test['f14']  =v14,t14,
all_train['f15'],all_test['f15']  =v15,t15,
all_train['f16'],all_test['f16']  =v16,t16,
all_train['f17'],all_test['f17']  =v17,t17,
all_train['f18'],all_test['f18']  =v18,t18,
all_train['f19'],all_test['f19']  =v19,t19,
all_train['f20'],all_test['f20']  =v20,t20,
all_train['f21'],all_test['f21']  =v21,t21,
all_train['f22'],all_test['f22']  =v22,t22,
all_train['f23'],all_test['f23']  =v23,t23,
all_train['f24'],all_test['f24']  =v24,t24,
all_train['f25'],all_test['f25']  =v25,t25,
all_train['f26'],all_test['f26']  =v26,t26,
all_train['f27'],all_test['f27']  =v27,t27,
all_train['f28'],all_test['f28']  =v28,t28,
all_train['f29'],all_test['f29']  =v29,t29,
all_train['f30'],all_test['f30']  =v30,t30,

all_train['f31'],all_test['f31']  =v31,t31,
all_train['f32'],all_test['f32']  =v32,t32,
all_train['f33'],all_test['f33']  =v33,t33
all_train['f34'],all_test['f34']  =v34,t34
all_train['f35'],all_test['f35']  =v35,t35
all_train['f36'],all_test['f36']  =v36,t36
all_train['f37'],all_test['f37']  =v37,t37
all_train['f38'],all_test['f38']  =v38,t38
all_train['f39'],all_test['f39']  =v39,t39
all_train['f40'],all_test['f40']  =v40,t40
all_train['f41'],all_test['f41']  =v41,t41
all_train['f42'],all_test['f42']  =v42,t42
all_train['f43'],all_test['f43']  =v43,t43
all_train['f44'],all_test['f44']  =v44,t44
all_train['f45'],all_test['f45']  =v45,t45

# all_train['f1'], all_test['f1']  = load_test_valid('nezha-base_0.9591154881244821')
# # all_train['f2'], all_test['f2']  = load_test_valid('nezha-large_0.970280295812428')
# all_train['f3'], all_test['f3'] = load_test_valid('nezha-base_0.9731846431788207')
# all_train['f4'], all_test['f4'] = load_test_valid('nezha-base_0.9744678930253807')
# # all_train['f5'], all_test['f5'] = load_test_valid('nezha-base_0.9748702884103849')
# # all_train['f6'], all_test['f6']  = load_test_valid('nezha-base_0.9748813528826525')
# # all_train['f7'], all_test['f7']  = load_test_valid('nezha-base_0.9750660632375358')
# # all_train['f8'], all_test['f8']  = load_test_valid('nezha-large_0.9681657612090724')
# # all_train['f8'], all_test['f8']  = load_test_valid('nezha-large_0.970280295812428')
# all_train['f9'], all_test['f9']  = load_test_valid('nezha-large_0.9722644834299695_test')


# all_train['f11'], all_test['f11']  = load_test_valid(0.7748161764705883)
# all_train['f12'], all_test['f12']  = load_test_valid(0.7794956745812627)
# all_train['f13'], all_test['f13']  = load_test_valid(0.7826952665399584)
# all_train['f14'], all_test['f14']  = load_test_valid(0.7878124429848568)
# all_train['f15'], all_test['f15']  = load_test_valid(0.786074047892551)
# all_train['f16'], all_test['f16']  = load_test_valid(0.7840989399293286)
# all_train['f17'], all_test['f17']  = load_test_valid(0.7709487225299266)


# #加上这些只有825
# all_train['f18'], all_test['f18']  = load_test_valid(0.7886281588447653)
# all_train['f19'], all_test['f19']  = load_test_valid(0.7897426701335963)
# all_train['f20'], all_test['f20']  = load_test_valid(0.7827964729170417)
# all_train['f21'], all_test['f21']  = load_test_valid(0.7857014871886758)
# all_train['f22'], all_test['f22']  = load_test_valid(0.7844938453058976)
# all_train['f23'], all_test['f23']  = load_test_valid(0.7804581742450537)

In [11]:
len_train = len(all_train)
concat_df = all_train.append(all_test).reset_index(drop=True)

/root/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [12]:
all_train

,q1,q2,label,id,concat,f1,f2,f3,f4,f5,...,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45
0,1 2 3 4 5 6 7,8 9 10 4 11,0,0,1 2 3 4 5 6 78 9 10 4 11,0.005461,0.002425,0.004642,0.001745,0.001258,...,2.801378e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.0,5.0,2.0
1,12 13 14 15,12 15 11 16,0,1,12 13 14 1512 15 11 16,0.310112,0.291970,0.225009,0.020243,0.068551,...,7.434514e-01,0.500000,0.285714,0.251976,0.657805,0.198446,0.158164,4.0,4.0,0.0
2,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,1,2,17 18 12 19 20 21 22 23 2412 23 25 6 26 27 19,0.992396,0.993766,0.995375,0.997968,0.998760,...,1.000000e+00,0.408248,0.242536,0.193649,0.065166,0.029619,0.020381,9.0,7.0,2.0
3,28 29 30 31 11,32 33 34 30 31,1,3,28 29 30 31 1132 33 34 30 31,0.978809,0.934222,0.986573,0.992213,0.990483,...,1.000000e+00,0.400000,0.333333,0.273861,0.629400,0.366676,0.250236,5.0,5.0,0.0
4,29 35 36 29,29 37 36 29,1,4,29 35 36 2929 37 36 29,0.931099,0.981896,0.956714,0.982109,0.996670,...,1.000000e+00,0.666667,0.547723,0.474342,0.609225,0.428979,0.324484,4.0,4.0,0.0
5,38 23 39 9 40,12 19 41 42 23 43 12 23 44 41 42 19,0,5,38 23 39 9 4012 19 41 42 23 43 12 23 44 41 42 19,0.124099,0.172333,0.312144,0.240001,0.033746,...,2.511391e-37,0.188982,0.091670,0.068041,0.066968,0.026713,0.018005,5.0,12.0,-7.0
6,45 46 47 48 49 50 45 48 51 32 52 53 54,55 46 47 48 49 56 57 58 59,0,6,45 46 47 48 49 50 45 48 51 32 52 53 5455 46 47...,0.001919,0.001984,0.001693,0.001036,0.000960,...,1.341844e-87,0.402015,0.361961,0.337053,0.310524,0.224089,0.187289,13.0,9.0,4.0
7,12 60 61 29,12 61 29,1,7,12 60 61 2912 61 29,0.976884,0.986472,0.989516,0.989460,0.998316,...,9.904386e-01,0.866025,0.730297,0.666667,0.819440,0.637381,0.550793,4.0,3.0,1.0
8,12 62 43 29 63 11,29 64 12 62 43 65 11 66,1,8,12 62 43 29 63 1129 64 12 62 43 65 11 66,0.994432,0.995967,0.993955,0.997576,0.998697,...,1.000000e+00,0.721688,0.544949,0.461880,0.423130,0.258924,0.199593,6.0,8.0,-2.0
9,29 13 67 12 68 69 70 16,71 10 72 29 68 69 70,1,9,29 13 67 12 68 69 70 1671 10 72 29 68 69 70,0.941838,0.863192,0.335851,0.928894,0.912988,...,1.000000e+00,0.534522,0.429669,0.378517,0.616281,0.372886,0.285430,8.0,7.0,1.0


In [13]:
for i in [f'f{i}' for i in range(1,45)]:
    try:
        concat_df = diff_num_helper(concat_df,num_col=i)
        concat_df = diff_num_helper2(concat_df,num_col=i)
        concat_df = pd.concat([concat_df, agg_helper(concat_df[['id']], concat_df,i ,'id')], axis=1)
    except:
        pass

In [14]:
# concat_df = pd.concat([concat_df, agg_helper2(concat_df[['id']], concat_df,'id_sub' ,'id')], axis=1)

In [15]:
# concat_df['sub_id_r'] = concat_df['AGG_nunique_test_id_sub_id']-concat_df['id_sub']

In [16]:
# concat_df['match_share'] = concat_df.apply(lambda x:word_match_share(x.q1,x.q2),axis=1)
# concat_df['match_share2'] = concat_df.apply(lambda x:levenshteinDistance(x.q1,x.q2),axis=1)
# concat_df['z_len1'] = concat_df.q1.map(lambda x: len(str(x)))
# concat_df['z_len2'] = concat_df.q2.map(lambda x: len(str(x)))
# concat_df['z_len'] = concat_df['z_len1'] + concat_df['z_len2'] 
# concat_df['z_lendiff'] = concat_df['z_len1'] - concat_df['z_len2']
# concat_df['z_lendiffabs'] = abs(concat_df['z_lendiff'])
# concat_df['z_ration'] = concat_df['z_len1'] /(concat_df['z_len2'] + 0.000000001) 
# concat_df['sub_id_r_ratio'] = concat_df['sub_id_r'] / concat_df['AGG_nunique_test_id_sub_id']
# concat_df['id_sub_ratio'] = concat_df['id_sub'] / concat_df['AGG_nunique_test_id_sub_id']
# concat_df['match_share_set'],concat_df['match_share_set1'] ,concat_df['match_share_set2']  = zip(*concat_df.apply(lambda x:word_match_share_set(x.q1,x.q2),axis=1))

In [17]:
concat_df = pd.concat([concat_df, agg_helper3(concat_df[['id']], concat_df,'id' ,'q1')], axis=1)
concat_df = pd.concat([concat_df, agg_helper3(concat_df[['id']], concat_df,'id' ,'q2')], axis=1)
concat_df['c'] = concat_df['q1'] +  concat_df['q2']
concat_df = pd.concat([concat_df, agg_helper3(concat_df[['id']], concat_df,'id' ,'c')], axis=1)

In [18]:
all_train = concat_df[:len_train]
all_test = concat_df[len_train:].reset_index(drop=True)
all_test['id'] = all_test['id']-1000000

In [19]:
train_df = all_train
all_target = ['label'] 
predictors = [i for i in train_df.columns if i not in all_target + ['id','id', 'q1', 'q2','c','concat']]
test_df = all_test
train_label = train_df[all_target]

In [20]:
train_df

,concat,f1,f10,f11,f12,f13,f14,f15,f16,f17,...,f8,f9,id,label,q1,q2,AGG_count_test_id_q1,AGG_count_test_id_q2,c,AGG_count_test_id_c
0,1 2 3 4 5 6 78 9 10 4 11,0.005461,0.007595,0.001062,0.002442,0.001540,0.003886,0.003231,0.005251,0.001428,...,0.003222,0.007682,0,0.0,1 2 3 4 5 6 7,8 9 10 4 11,1,8,1 2 3 4 5 6 78 9 10 4 11,1
1,12 13 14 1512 15 11 16,0.310112,0.288911,0.323457,0.527039,0.450285,0.125232,0.473448,0.035396,0.050406,...,0.137944,0.252190,1,0.0,12 13 14 15,12 15 11 16,2,1,12 13 14 1512 15 11 16,1
2,17 18 12 19 20 21 22 23 2412 23 25 6 26 27 19,0.992396,0.993460,0.998446,0.994700,0.994971,0.998885,0.995327,0.997712,0.996726,...,0.996543,0.995877,2,1.0,17 18 12 19 20 21 22 23 24,12 23 25 6 26 27 19,2,1,17 18 12 19 20 21 22 23 2412 23 25 6 26 27 19,1
3,28 29 30 31 1132 33 34 30 31,0.978809,0.890139,0.986739,0.925759,0.962047,0.995044,0.831900,0.996465,0.969656,...,0.979201,0.946196,3,1.0,28 29 30 31 11,32 33 34 30 31,3,1,28 29 30 31 1132 33 34 30 31,1
4,29 35 36 2929 37 36 29,0.931099,0.973825,0.970398,0.899474,0.933754,0.993720,0.983519,0.982641,0.986341,...,0.953600,0.950936,4,1.0,29 35 36 29,29 37 36 29,1,1,29 35 36 2929 37 36 29,1
5,38 23 39 9 4012 19 41 42 23 43 12 23 44 41 42 19,0.124099,0.251779,0.022536,0.276653,0.075835,0.495468,0.197836,0.027047,0.041218,...,0.408012,0.365695,5,0.0,38 23 39 9 40,12 19 41 42 23 43 12 23 44 41 42 19,1,1,38 23 39 9 4012 19 41 42 23 43 12 23 44 41 42 19,1
6,45 46 47 48 49 50 45 48 51 32 52 53 5455 46 47...,0.001919,0.002009,0.000897,0.002021,0.001485,0.000726,0.001564,0.001105,0.001672,...,0.001887,0.002022,6,0.0,45 46 47 48 49 50 45 48 51 32 52 53 54,55 46 47 48 49 56 57 58 59,1,1,45 46 47 48 49 50 45 48 51 32 52 53 5455 46 47...,1
7,12 60 61 2912 61 29,0.976884,0.981626,0.984295,0.964306,0.969626,0.997436,0.983410,0.996970,0.993633,...,0.989750,0.982099,7,1.0,12 60 61 29,12 61 29,1,2,12 60 61 2912 61 29,1
8,12 62 43 29 63 1129 64 12 62 43 65 11 66,0.994432,0.988983,0.998172,0.987023,0.992712,0.997595,0.995785,0.998274,0.998319,...,0.995118,0.992511,8,1.0,12 62 43 29 63 11,29 64 12 62 43 65 11 66,4,1,12 62 43 29 63 1129 64 12 62 43 65 11 66,1
9,29 13 67 12 68 69 70 1671 10 72 29 68 69 70,0.941838,0.710409,0.958534,0.681237,0.796243,0.325580,0.680602,0.933728,0.988839,...,0.791659,0.839576,9,1.0,29 13 67 12 68 69 70 16,71 10 72 29 68 69 70,1,1,29 13 67 12 68 69 70 1671 10 72 29 68 69 70,1


In [21]:
len(predictors)

48

In [22]:
predictors

['f1',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f2',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f25',
 'f26',
 'f27',
 'f28',
 'f29',
 'f3',
 'f30',
 'f31',
 'f32',
 'f33',
 'f34',
 'f35',
 'f36',
 'f37',
 'f38',
 'f39',
 'f4',
 'f40',
 'f41',
 'f42',
 'f43',
 'f44',
 'f45',
 'f5',
 'f6',
 'f7',
 'f8',
 'f9',
 'AGG_count_test_id_q1',
 'AGG_count_test_id_q2',
 'AGG_count_test_id_c']

In [23]:
def search_f1(y_true, y_pred):
    best = 0
    best_t = 0
    for i in range(1,200):
        tres = i / 200
        y_pred_bin =  (y_pred > tres).astype(int)
        score = f1_score(y_true, y_pred_bin)
        if score > best:
            best = score
            best_t = tres
    print('best', best)
    print('thres', best_t)
    return best, best_t

In [28]:
def display_importances(feature_importance_df_,score=0):
    ft = feature_importance_df_[["feature", "split","gain"]].groupby("feature").mean().sort_values(by="gain", ascending=False)
    print(ft.head(60))
    print("_______________last_____________________")
    print(ft.tail(60))

def lgbstack():
    sub_result = []
    scores = []
    scores2 = []
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    folds_l=[]
    feature_importance_df = pd.DataFrame()
    for target in all_target:
        target = [target]

        num_boost_round = 2000
        early_stopping_rounds = 600
        verbose_eval = 300
        lgb_params =  {
            'nthread':10,
            'learning_rate':0.01,
            'num_leaves':32,
            'colsample_bytree':0.4,
            'subsample':0.65,
            'max_depth':-1,
            'objective':'binary',
            'metric':'binary,auc',
            'seed':2018,
            'verbose':-1
            }
        kfold = KFold(n_splits=5, shuffle=False, random_state=1024)
#         kfold = KFold(n_splits=5,random_state=42)
        index = kfold.split(train_df)
#         folds = GroupKFold(n_splits=5).split(X=train_df.q2, groups=train_df.q1)
        perm = [i for i in range(len(train_df))]
        perm = pd.DataFrame(perm)
        perm.columns = ['index_']
        for n_fold, (train_idx, valid_idx) in enumerate(index): 
            print("fold_index:", n_fold)
            print('train_idx',train_idx)
            print('train_idx',valid_idx)
            #continue
            train_x, train_y = train_df[predictors].iloc[train_idx], train_label[target].iloc[train_idx]
            valid_x, valid_y = train_df[predictors].iloc[valid_idx], train_label[target].iloc[valid_idx]
            xgtrain = lgb.Dataset(train_x, label=train_y,
                                  feature_name=predictors,
                                  )
            xgvalid = lgb.Dataset(valid_x, label=valid_y,
                                  feature_name=predictors,
                                  )

            clf = lgb.train(lgb_params, 
                             xgtrain, 
                             valid_sets=[xgvalid], 
                             valid_names=['valid'], 
                             num_boost_round=num_boost_round,
                             early_stopping_rounds=early_stopping_rounds,
                             verbose_eval=verbose_eval)

            oof_preds[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
            fold_preds =  clf.predict(test_df[predictors], num_iteration=clf.best_iteration)
            sub_preds += fold_preds / 5#folds.n_splits
            folds_l.append(fold_preds)
            

            gain = clf.feature_importance('gain')
            fold_importance_df = pd.DataFrame({'feature':clf.feature_name(),
                                               'split':clf.feature_importance('split'),
                                               'gain':100*gain/gain.sum(),
                                               'fold':n_fold,                        
                                               }).sort_values('split',ascending=False)
            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
            #print('Fold %2d AUC : %.6f' % (n_fold + 1, np.sqrt(mean_squared_error(valid_y, oof_preds[valid_idx]))))
            gc.collect()
            s= roc_auc_score(valid_y,oof_preds[valid_idx] )
            scores2.append(s)
        scores.append(log_loss(train_label[target],oof_preds)) 
        display_importances(feature_importance_df)
        
    return oof_preds, sub_preds,folds_l


In [29]:
#test
oof_preds, sub_preds,folds = lgbstack()

/root/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


fold_index: 0
train_idx [20000 20001 20002 ... 99997 99998 99999]
train_idx [    0     1     2 ... 19997 19998 19999]
Training until validation scores don't improve for 600 rounds
[300]	valid's binary_logloss: 0.177221	valid's auc: 0.982011
[600]	valid's binary_logloss: 0.165395	valid's auc: 0.982171
[900]	valid's binary_logloss: 0.165557	valid's auc: 0.982211
[1200]	valid's binary_logloss: 0.16595	valid's auc: 0.98223
Early stopping, best iteration is:
[703]	valid's binary_logloss: 0.16507	valid's auc: 0.982205
fold_index: 1
train_idx [    0     1     2 ... 99997 99998 99999]
train_idx [20000 20001 20002 ... 39997 39998 39999]
Training until validation scores don't improve for 600 rounds
[300]	valid's binary_logloss: 0.17482	valid's auc: 0.9827
[600]	valid's binary_logloss: 0.161583	valid's auc: 0.982908
[900]	valid's binary_logloss: 0.160888	valid's auc: 0.982944
[1200]	valid's binary_logloss: 0.160752	valid's auc: 0.982966
[1500]	valid's binary_logloss: 0.160787	valid's auc: 0.98296

In [26]:
oof_preds

array([0.00364917, 0.17129155, 0.9961245 , ..., 0.99856966, 0.45870467,
       0.99124807])

In [31]:
roc_auc_score(train_label['label'],oof_preds)

0.982459215047588

In [30]:
best_score = roc_auc_score(train_label['label'],oof_preds)
#0.9788849490690293
#0.981430721703091
#0.9820907196233832
#0.9823898040988547

In [28]:
np.save(f'B/oof_id_stacking_{best_score}.npy',oof_preds)

In [27]:
sub_preds[0]

0.1143981438524819

In [30]:
all_train, all_test = load_train()
all_test['label'] = same['label']
best_score = roc_auc_score(train_label['label'],oof_preds)
print(best_score)
np.save(f'B/esim/stacking_oof_{best_score}.npy',oof_preds)
np.save(f'B/esim/stacking_sub_{best_score}.npy',sub_preds)
#np.save(f'beikeoof/stacking__pl_{best_score}.npy',folds)
sub = sub_preds#np.average(test_preds, axis=0) 
F = open(f'B/esim/stacking_oof_{best_score}.txt', 'w')
for i in tqdm(range(len(sub_preds))):
    #y_pred = model.predict(x_true)[:, 0, 5:7]
    #y_pred = y_pred[:, 1] / (y_pred.sum(axis=1) + 1e-8)
    F.write('%f\n' % sub_preds[i])
F.close()
# all_test[['id','id_sub','label']].to_csv('esim/esim/stacking_beike_{}.csv'.format(best_score),index=False, header=None,sep='\t')

100%|██████████| 25000/25000 [00:00<00:00, 827853.66it/s]

0.9823898040988547


In [ ]:
np.save(f'torch_nezha_large50_mingwen/oof_id_{modelname}_{best_score}_test.npy',train_preds)
np.save(f'torch_nezha_large50_mingwen/sub_id_{modelname}_{best_score}_test.npy',test_preds)
sub = test_preds#np.average(test_preds, axis=0) 
sub = sub > 0.5
df_test['label'] = sub.astype(int)
# df_test[['id','label']].to_csv('torch_nezha/submission_beike_{}.csv'.format(best_score),index=False, header=None,sep='\t')

F = open(f'torch_nezha_large50_mingwen/sub_id_{modelname}_{best_score}_test.txt', 'w')
for x_true in tqdm(test_preds):
    #y_pred = model.predict(x_true)[:, 0, 5:7]
    #y_pred = y_pred[:, 1] / (y_pred.sum(axis=1) + 1e-8)
    for p in x_true:
        F.write('%f\n' % p)